# Семинар 5: Нейросетевое ранжирование
### Семинарист: Матвеев Артем, Yandex

В этом семинаре мы познакомимся с методами кодирование входных признаков для моделей нейросетевого ранжирвания: Mutisize Encoding with Unified Embeddings для категориальных признаков и Picewise Linear Encoding для вещественных. А также имплементируем DCNv2 и проверим всю схему на наборе данных Criteo Kaggle Dataset. Попробуем обогнать такой сильный бейзлайн на табличных данных как Catboost.

In [2]:
# !pip install -r requirements.txt

In [3]:
import typing as tp
import polars as pl
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score

## 1. Downsampled Criteo Kaggle Dataset
Данные лежат здесь: https://www.kaggle.com/datasets/dogrose/downsampled-criteo-kaggle-dataset/data.

Criteo Dataset — это крупномасштабный набор данных для задач предсказания кликов (Click-Through Rate, CTR) в рекламе. Он используется для разработки и тестирования алгоритмов, прогнозирующих вероятность клика пользователя на рекламное объявление.

**Особенности датасета:**
- Содержит миллионы рекламных показов
- Включает бинарную целевую переменную (клик/не клик)
- 13 числовых признаков (I1-I13), представляющих различные счётчики
- 26 категориальных признаков (C1-C26), анонимизированных и хэшированных
- Данные разделены на обучающую (6 дней) и тестовую (1 день) выборки
- Версия Downsampled Criteo Kaggle Dataset была уменьшена в 10 раз относительно оригинала.

Оригинальный датасет содержит множество пропущенных значений и требует тщательной предобработки, которая реализована в предоставленном коде.

Проводилось одноименное соревнование на Kaggle: https://www.kaggle.com/c/criteo-display-ad-challenge/overview. Наиболее успешные решения в соревнованиях Criteo на Kaggle обычно включали:

**Методы предобработки данных:**
- Логарифмическая трансформация числовых признаков (как реализовано в коде)
- Count encoding или one-hot encoding для категориальных признаков
- Feature hashing для категориальных признаков с высокой кардинальностью
- Создание комбинированных признаков (feature interactions)

**Алгоритмы:**
- Градиентный бустинг: XGBoost, LightGBM, CatBoost
- Глубокие нейронные сети, особенно Wide & Deep модели
- Факторизационные машины (Field-aware Factorization Machines, FFM)
- Ансамбли различных моделей

Победители часто использовали комбинацию этих подходов, уделяя особое внимание инженерии признаков и обработке категориальных переменных с высокой кардинальностью.

In [4]:
# !mkdir ./data
# !curl -L -o ./data/downsampled-criteo-kaggle-dataset.zip\
#   https://www.kaggle.com/api/v1/datasets/download/dogrose/downsampled-criteo-kaggle-dataset
# !unzip ./data/downsampled-criteo-kaggle-dataset.zip -d ./data

In [5]:
class CriteoDatasetUtils:
    INT_COLS = [f'I{i + 1}' for i in range(13)]
    CAT_COLS = [f'C{i + 1}' for i in range(26)]
    LABEL_COl = 'label'

    @classmethod
    def preprocess_dense_features(cls, lf: pl.LazyFrame) -> pl.LazyFrame:
        """
        Preprocess dense features:
        - Fill missing values with 0
        - Apply log transformation: log(x+1) or log(x+4)
        """
        expressions = []
        for col in cls.INT_COLS:
            expressions.append(
                pl.col(col).fill_null(0).add(1 if col != 'I2' else 4).log()
            )
        lf = lf.with_columns(expressions)
        return lf

    @classmethod
    def preprocess_categorical_features(cls, lf: pl.LazyFrame) -> pl.LazyFrame:
        """
        Preprocess categorical features:
        - Fill missing values with zero string ("00000000")
        - Convert from hex to Int64
        """
        expressions = []
        for col in cls.CAT_COLS:
            expressions.append(
                pl.col(col).fill_null("00000000").str.to_integer(base=16)
            )
        lf = lf.with_columns(expressions)
        return lf

    @classmethod
    def read_and_preprocess(cls, path: str) -> pl.DataFrame:
        lf = pl.scan_parquet(path)
        lf = cls.preprocess_categorical_features(lf)
        lf = cls.preprocess_dense_features(lf)
        return lf.collect()

In [6]:
# DATASETS_PATH = '/content'
DATASETS_PATH = './data'
train_df = CriteoDatasetUtils.read_and_preprocess(f'{DATASETS_PATH}/criteo_train_6days_downsampled.parquet')
test_df = CriteoDatasetUtils.read_and_preprocess(f'{DATASETS_PATH}/criteo_test_1day_downsampled.parquet')
train_df.head(5)

label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,0.693147,1.609438,1.791759,0.0,7.23201,1.609438,2.772589,1.098612,5.204007,0.693147,1.098612,0.0,1.098612,1761418852,2162322587,4220739894,2068259780,633879704,2114768079,3732510136,529118562,2805916944,2832028932,2999688344,935969124,673490422,450684655,2343089050,2300273383,3854202482,4114618041,568184265,2972002973,129309004,0,974593739,3318023300,3904386055,2535972118
0,0.0,1.791759,6.45047,0.0,10.946781,0.0,0.0,1.791759,4.189655,0.0,0.0,0.0,1.098612,98275684,73979506,2062028047,2161661274,633879704,2114768079,69696505,185940084,2093428418,990438539,2116836373,3486017542,2443294225,2995026422,1478826667,342520061,2003624857,187896596,568184265,1480633834,3421256155,0,974593739,3470446969,3935970412,2589289724
1,0.0,3.931826,0.0,0.0,8.764053,3.663562,2.995732,2.397895,4.969813,0.0,2.397895,0.0,1.94591,342162125,950618017,574295574,3394569756,633879704,2114768079,1765007041,1530472565,2805916944,990438539,2248945707,359635439,812864628,450684655,242755870,1606371972,3854202482,429505257,0,0,3736690781,0,851920782,3829933919,0,0
1,0.0,5.638355,0.0,1.386294,8.898229,3.218876,1.94591,1.386294,4.59512,0.0,0.693147,0.0,1.386294,2364568165,2598325497,779549537,186309082,1291264903,4268462821,1977395914,185940084,2805916944,990438539,2326518504,2558959783,3989772238,131152527,716094755,864816393,3854202482,3353078997,0,0,1624430225,0,974593739,2427856751,0,0
0,0.0,3.73767,1.098612,1.609438,8.045588,5.010635,4.174387,3.89182,4.941642,0.0,1.94591,1.94591,1.609438,98275684,648577312,3492987491,3247169921,1291264903,4222442646,1062127239,529118562,2805916944,1919877373,3299735832,3753576955,2245779768,131152527,68076599,2223606570,2399067775,1465486885,0,0,1360682,0,851920782,991444060,0,0


In [9]:
assert train_df.null_count().pipe(sum).item() == 0
assert test_df.null_count().pipe(sum).item() == 0

Смотрим на количество уникальных значений категориальных признаков.

In [10]:
unique_counts = {col: train_df[col].n_unique() for col in CriteoDatasetUtils.CAT_COLS}
sorted_unique_counts = dict(
    sorted(unique_counts.items(), key=lambda item: item[1], reverse=True)
)
sorted_unique_counts

{'C3': 1203747,
 'C12': 1047892,
 'C21': 925198,
 'C16': 759151,
 'C4': 378887,
 'C24': 90428,
 'C26': 59490,
 'C10': 50127,
 'C7': 11950,
 'C15': 11775,
 'C11': 5215,
 'C18': 4756,
 'C13': 3164,
 'C19': 2036,
 'C1': 1452,
 'C8': 628,
 'C2': 556,
 'C5': 303,
 'C25': 94,
 'C14': 26,
 'C6': 18,
 'C22': 17,
 'C23': 15,
 'C17': 10,
 'C20': 4,
 'C9': 3}

Посчитаем необходимое количество GPU памяти.

In [11]:
uniq_ids = sum(sorted_unique_counts.values())
embedding_dim = 256
bytes_in_float = 4
mult = 4 # params + grads + moment1 + moment2
bytes_in_gb = 1024 * 1024 * 1024 
print(f'{uniq_ids * embedding_dim * bytes_in_float * mult / bytes_in_gb} GB')

17.38335418701172 GB


## 2. Multisize Unified Embeddings

Для кодирования категориальных признаков будем использовать Multisize Unified кодирование от Google DeepMind: [Unified Embedding: Battle-Tested Feature Representations for Web-Scale ML Systems](https://arxiv.org/abs/2305.12102).

### Общая задача

Дан $D = \{(x_1, y_1), (x_2, y_2), \ldots, (x_{|D|}, y_{|D|})\}$ с примерами из $T$ категориальных признаков с словарями $\{V_1, V_2, \ldots, V_T\}$. Каждый пример $x = [v_1, v_2, \ldots, v_T]$, где $v_i \in V_i$.

- Матрица эмбеддингов $\mathbf{E} \in \mathbb{R}^{M \times d}$, отображения примера в эмбеддинг $g(\mathbf{x}; \mathbf{E})$. 
- Хеш-функция $h(v) : V \rightarrow [M]$ назначает значение признака индексу строки (используется в $g(\mathbf{x}; \mathbf{E})$).
- Функция модели $f(\mathbf{e}; \boldsymbol{\theta})$ преобразует вложения в предсказание.

Задача обучения:
$$\arg \min_{\mathbf{E}, \boldsymbol{\theta}} \mathcal{L}_D(\mathbf{E}, \boldsymbol{\theta}), \quad \text{где} \quad \mathcal{L}_D(\mathbf{E}, \boldsymbol{\theta}) = \sum_{(\mathbf{x},y) \in D} \ell(f(g(\mathbf{x}; \mathbf{E}); \boldsymbol{\theta}), y).$$

Используем $h_t(v)$ для каждого признака $t \in [T]$. Обозначаем $\mathbf{e}_m$ для $m$-й строки $\mathbf{E}$, и $\mathbb{1}_{u,v}$ как индикатор коллизии хешей между $u$ и $v$.

### Как это работает

Далее будем предполагать, что $|T|$ = 2.

<div style="width:90%; margin: auto;">

![](https://i.ibb.co/GKMKcvm/unified-embeddings.png)

</div>


### Почему это работает (интуиция)

Рассмотрим частный случай (решаем бинарную классфикацию с помощью логистической регрессии):

$$y_i \in \{0, 1\}$$
$$ D_0 = \{(x_i, y_i) \in D : y_i = 0\} $$
$$ D_1 = \{(x_i, y_i) \in D : y_i = 1\} $$
$$ C_{u,v,0} = |\{([u, v], y) \in D : y = 0\}|$$
$$ \sigma_\theta(z) = \frac{1}{1 + \exp(-\langle z, \theta \rangle)} $$
$$ z = g(x; \mathbf{E}) = [e_{h_1(x_1)}, e_{h_2(x_2)}] $$
$$ \theta = [\theta_1, \theta_2],~\theta_t \in \mathbb{R}^M$$


Функция потерь бинарной кросс-энтропии:
$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{(x,y)\in D_0} \log \left( \frac{1}{1 + \exp(-\langle \theta, g(x; \mathbf{E}) \rangle)} \right) - \sum_{(x,y)\in D_1} \log \left( \frac{1}{1 + \exp(\langle \theta, g(x; \mathbf{E}) \rangle)} \right) $$

Перепишем функция потерь (через частоты совместного появления):
$$ e_{u,v} = [e_{h_1(u)}, e_{h_2(v)}] $$

$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{u\in V_1} \sum_{v\in V_2} C_{u,v,0} \log \left( \frac{1}{1 + \exp(-\theta^\top e_{u,v})} \right) + C_{u,v,1} \log \left( \frac{1}{1 + \exp(\theta^\top e_{u,v})} \right) $$

После объединения сигмоидных функций:
$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{u\in V_1} \sum_{v\in V_2} C_{u,v,0} \log \exp(\theta^\top e_{u,v}) - (C_{u,v,0} + C_{u,v,1}) \log(1 + \exp(\theta^\top e_{u,v})) $$

Далее будем предполагать, что обучаем наш алгоритм с SGD. Посчитаем градиенты по эмбеддингам. Полный градиент для эмбеддинга с учетом внутри- и межпризнаковых взаимодействий:
$$ \nabla_{E_{h(u)}} \mathcal{L}_D(\mathbf{E}, \theta) = $$
$$ \theta_1 \sum_{v\in V_2} C_{u,v,0} - (C_{u,v,0} + C_{u,v,1})\sigma_\theta(e_{u,v}) \tag{1}$$
$$ + \theta_1 \sum_{w\in V_1, w\neq u} \mathbb{1}_{u,w} \sum_{v\in V_2} C_{w,v,0} - (C_{w,v,0} + C_{w,v,1})\sigma_\theta(e_{u,v}) \tag{2}$$
$$ + \theta_2 \sum_{v\in V_2} \mathbb{1}_{u,v} \sum_{w\in V_1} C_{w,v,0} - (C_{w,v,0} + C_{w,v,1})\sigma_\theta(e_{w,u}) \tag{3}$$

Анализируем:
- $(1)$ collisionless компонента.
- $(2)$ intra-feature компонента.
- $(3)$ inter-feature компонента.
- Компоненты $(2)$ и $(3)$ смещают реальный градиент.
- Intra-feature bias сонаправлен с collisionless компонентой, поэтому модель не может убрать это смещение.
- В случае SGD inter-feature bias может невелирован за счет $\theta_1$ ортогонально $\theta_2$, т.к. во время SGD, $e_{h(u)}$ представляет собой линейную комбинацию градиентов по шагам обучения, что означает, что $e_{h(u)}$ может быть разложено на компоненты в направлении $\theta_1$ и inter-feature компоненты в направлении $\theta_2$. Поскольку $\langle\theta_1, \theta_2\rangle = 0$, проекция $\theta_1^\top e_{h(u)}$ эффективно устраняет inter-feature  компонент.

<div style="width:70%; margin: auto;">

![](https://i.ibb.co/xt6nbrZ3/theory-unified.png)

</div>

Вывод: 

Не все коллизии одинаково проблематичны. Межпризнаковые коллизии могут быть смягчены однослойной нейронной сетью, поскольку разные признаки обрабатываются разными параметрами модели. Коллизии в рамках одного признака убираем за счет нескольких взятий хешей.

In [12]:
class MultihashTransform:
    """
    Applys transformation to training sample
    """
    def __init__(self, cardinality, seeds=None, name='sparse'):
        assert seeds is not None
        self._cardinality = cardinality
        self._name = name
        self._seeds = torch.tensor(seeds)

    def __call__(self, sample: dict[str, tp.Any]) -> dict[str, tp.Any]:
        sample[self._name] = (
            (sample[self._name].unsqueeze(1) + self._seeds) % self._cardinality
        ).long().reshape(-1)
        return sample

In [13]:
seeds = [
    [2342 + 13 * i, 7777 + 17 * i]
    for i in range(26)
]
transform = MultihashTransform(10, seeds)
input = {
    'label': torch.tensor(1),
    'dense': torch.randn(13),
    'sparse': torch.arange(26)
}
output = transform(input)
assert output['sparse'].shape == (2 * 26,)

In [15]:
print(output['sparse'])

tensor([2, 7, 6, 5, 0, 3, 4, 1, 8, 9, 2, 7, 6, 5, 0, 3, 4, 1, 8, 9, 2, 7, 6, 5,
        0, 3, 4, 1, 8, 9, 2, 7, 6, 5, 0, 3, 4, 1, 8, 9, 2, 7, 6, 5, 0, 3, 4, 1,
        8, 9, 2, 7])


In [14]:
class UnifiedEmbeddings(nn.Module):
    def __init__(self, cardinality, embedding_dim):
        super().__init__()
        self._cardinality = cardinality
        self._embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(
            num_embeddings=cardinality, embedding_dim=embedding_dim
        )

    def forward(self, ids: torch.tensor):
        # ids shape: [batch_size, num_features]
        return self.embeddings(ids)

## 3. Picewise Linear Encoding

Для кодирования вещественных признаков будем использовать Picewise Linear Encoding от Yandex Research: [On Embeddings for Numerical Features in Tabular Deep Learning](https://arxiv.org/abs/2203.05556).

GitHub: https://github.com/yandex-research/rtdl-num-embeddings.

<div style="width:90%; margin: auto;">

![](https://i.ibb.co/XZtk6fSN/picewise-linear.png)

</div>

Эмбеддинги числовых признаков формализуются как $z_i = f_i(x_i^{(num)}) \in \mathbb{R}^{d_i}$, где:
- $f_i(x)$ — функция эмбеддинга для i-го числового признака
- $z_i$ — результирующий вектор эмбеддинга
- $d_i$ — размерность эмбеддинга

Ключевые особенности:
- Эмбеддинги вычисляются независимо для каждого признака
- В MLP-архитектурах эмбеддинги конкатенируются в один вектор
- В Transformer-архитектурах эмбеддинги используются без дополнительных преобразований

**Кусочно-линейное кодирование (PLE)**

PLE разбивает диапазон значений числового признака на $T$ интервалов (бинов) $B_1, \ldots, B_T$ с границами $[b_0, b_1], [b_1, b_2], ..., [b_{T-1}, b_T]$.

Формальное определение: $\text{PLE}(x) = [e_1, \ldots, e_T] \in \mathbb{R}^T$

где компоненты $e_t$ вычисляются как:

$$
e_t = 
\begin{cases}
0, & \text{если } x < b_{t - 1} \text{ И } t > 1 \\
1, & \text{если } x \geq b_t \text{ И } t < T \\
\frac{x-b_{t-1}}{b_t-b_{t-1}}, & \text{иначе}
\end{cases}
$$

Важные свойства:

- При $T = 1$ PLE эквивалентно скалярному представлению
- В отличие от категориальных признаков, PLE учитывает упорядоченность числовых данных
- PLE можно рассматривать как предобработку признаков

Применение в моделях с вниманием:

В моделях с механизмом внимания требуется дополнительно учитывать информацию об индексах признаков:

1. Для каждого бина $B_t$ выделяется обучаемый эмбеддинг $v_t \in \mathbb{R}^d$
2. Итоговый эмбеддинг вычисляется как: $f_i(x) = v_0 + \sum_{t=1}^T e_t \cdot v_t = \text{Linear}(\text{PLE}(x))$

Построение бинов:

Наиболее распространенный подход к построению бинов — разбиение по квантилям эмпирического распределения числового признака. Формально:
- Для i-го признака: $b_t = q_t \left(\{x_j^{(num)}\}_{j \in J_{train}}\right)$, где $q$ — функция эмпирического квантиля


In [16]:
class PiecewiseLinearEncodingTransform:
    """
    Applys transformation to training sample
    """
    @staticmethod
    def compute_bins(
        X: torch.Tensor,
        n_bins: int,
    ) -> list[torch.Tensor]:
        bins = [
            q.unique()
            for q in torch.quantile(
                X, torch.linspace(0.0, 1.0, n_bins + 1).to(X), dim=0
            ).T
        ]
        return bins

    def __init__(self, dense_train_df, n_bins=32, name='dense'):
        self._name = name
        self._bins = PiecewiseLinearEncodingTransform.compute_bins(dense_train_df.to_torch(), n_bins)
        n_features = len(self._bins)
        self._n_bins = [len(x) - 1 for x in self._bins]
        max_n_bins = max(self._n_bins)

        self.weight = torch.zeros(n_features, max_n_bins)
        self.bias = torch.zeros(n_features, max_n_bins)

        for i, bin_edges in enumerate(self._bins):
            bin_width = bin_edges.diff()
            w = 1.0 / bin_width
            b = -bin_edges[:-1] / bin_width
            self.weight[i, -1] = w[-1]
            self.bias[i, -1] = b[-1]
            self.weight[i, :self._n_bins[i] - 1] = w[:-1]
            self.bias[i, :self._n_bins[i] - 1] = b[:-1]
    
    @property
    def n_bins(self):
        return self._n_bins

    def __call__(self, sample: dict[str, tp.Any]) -> dict[str, tp.Any]:
        x = sample[self._name].to(torch.float32).unsqueeze(0)
        x = torch.addcmul(self.bias, self.weight, x[..., None])
        x = torch.cat(
            [
                x[..., :1].clamp_max(1.0),
                x[..., 1:-1].clamp(0.0, 1.0),
                x[..., -1:].clamp_min(0.0)
            ],
            dim=-1,
        )
        x = x.flatten(-2).squeeze(0)
        sample[self._name] = x

        return sample

Пример:

**weight** = 
\begin{bmatrix}
\frac{1}{b_1 - b_0} & \frac{1}{b_2 - b_1} & \frac{1}{b_3 - b_2} & \frac{1}{b_4 - b_3} \\
\frac{1}{c_1 - c_0} & \frac{1}{c_2 - c_1} & \frac{1}{c_3 - c_2} & \frac{1}{c_4 - c_3}
\end{bmatrix}

**bias** = 
\begin{bmatrix}
-\frac{b_0}{b_1 - b_0} & -\frac{b_1}{b_2 - b_1} & -\frac{b_2}{b_3 - b_2} & -\frac{b_3}{b_4 - b_3} \\
-\frac{c_0}{c_1 - c_0} & -\frac{c_1}{c_2 - c_1} & -\frac{c_2}{c_3 - c_2} & -\frac{c_3}{c_4 - c_3}
\end{bmatrix}


In [17]:
transform = PiecewiseLinearEncodingTransform(train_df[CriteoDatasetUtils.INT_COLS], name='dense') 
input = {
    'label': torch.tensor(1),
    'dense': torch.randn(13),
    'sparse': torch.arange(26)
}
output = transform(input)
assert output['dense'].shape == (403,)

In [18]:
class PiecewiseLinearEncoding(nn.Identity):
    pass

## 4. DCN v2 - deep cross network

Для агрегации категориальных и вещественных признаков в один скаляр будем использовать DCNv2 от Google DeepMind: [DCN V2: Improved Deep & Cross Network and Practical Lessons for Web-scale Learning to Rank Systems](https://arxiv.org/abs/2008.13535).

### Подход

<div style="width:50%; margin: auto;">

![](https://i.ibb.co/ZqfF5yf/dcn-v2.png)
![](https://i.ibb.co/SDYWNSMy/dcn-v2-equation.png)

</div>

Stacked вариант:
- Сначала последовательность кросс-слоев: $$x_{i+1} = x_0 \odot (W \times x_i + b) + x_i.$$

- Затем последовательность Deep слоев: $$h_{l+1} = f(W_lh_l + b_l).$$


### Почему работает и зачем
- Значем, что cross признаки важны. 
- DNN выучивает только неявные взаимодейтсвия, плохо аппроксимирует dot-product => нужные глубокие сети. 
- CrossNet добавляет явные взаимодействия признаков => не нужны глубоки DNN => может применять в рантайме.

<div style="width:50%; margin: auto;">

![](https://i.ibb.co/K34HqJH/dcn-theory.png)

</div>

In [ ]:
class CrossLayer(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)

    def forward(self, x0, xl):
        return x0 * self.linear(xl) + xl


class CrossNetwork(torch.nn.Module):
    def __init__(self, input_dim, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(num_layers)])

    def forward(self, x):
        xl = x
        for layer in self.layers:
            xl = layer(x, xl)
        return xl


class DeepNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_units):
        super().__init__()
        layers = []
        for units in hidden_units:
            layers.append(nn.Linear(input_dim, units))
            layers.append(nn.ReLU())
            input_dim = units
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class DCNV2(nn.Module):
    def __init__(self, embedding_size, cross_layers, deep_units, input_size, cardinality=65536):
        super().__init__()
        self.sparse_encode_layer = UnifiedEmbeddings(cardinality, embedding_size)
        self.dense_encode_layer = PiecewiseLinearEncoding() 
        self.cross_network = CrossNetwork(input_size, cross_layers)
        self.deep_network = DeepNetwork(input_size, deep_units)
        self.output_layer = nn.Linear(deep_units[-1], 1)

    def forward(self, dense_input, sparse_input):
        sparse_embeddings = self.sparse_encode_layer(sparse_input).view(sparse_input.size(0), -1)
        dense_embeddings = self.dense_encode_layer(dense_input)
        combined_input = torch.cat([dense_embeddings, sparse_embeddings], dim=-1)
        cross_output = self.cross_network(combined_input)
        deep_output = self.deep_network(cross_output)
        return self.output_layer(deep_output).squeeze(dim=-1)

## 5. Обучаем нейросетевое ранжирование

In [20]:
class CriteoDataset(Dataset):
    def __init__(
            self,
            df: pl.DataFrame,
            transforms: list[tp.Callable[[tp.Any], tp.Any]] = None,
    ):
        self._labels = df[CriteoDatasetUtils.LABEL_COl].to_torch().to(torch.float32)
        self._dense = df[CriteoDatasetUtils.INT_COLS].to_torch()
        self._sparse = df[CriteoDatasetUtils.CAT_COLS].to_torch()
        self._transforms = (
            transforms if transforms is not None else []
        )

    def __len__(self):
        return self._labels.size(0)

    def __getitem__(self, idx):
        sample = {
            'label': self._labels[idx],
            'dense_features': self._dense[idx],
            'sparse_features': self._sparse[idx]
        }
        for transform in self._transforms:
            sample = transform(sample)
        return sample

In [21]:
batch_size = 4096
cardinality = 8 * 65536
seeds = [[2342 + 13 * i, 7777 + 17 * i, 131 + 833 * i] for i in range(len(CriteoDatasetUtils.CAT_COLS))]
num_hashes = 3
embedding_size = 64 
n_bins = 39 

In [22]:
dense_transform = PiecewiseLinearEncodingTransform(train_df[CriteoDatasetUtils.INT_COLS], n_bins, name='dense_features')
sparse_transform = MultihashTransform(cardinality, seeds, name='sparse_features')
transforms = [dense_transform, sparse_transform]

train_dataset = CriteoDataset(train_df, transforms)
val_dataset = CriteoDataset(test_df, transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [23]:
def train_model(model, train_loader, val_loader, epochs=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader):
            int_features, cat_features, labels = batch['dense_features'].to(device), batch['sparse_features'].to(device), batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(int_features, cat_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss, correct, total = 0, 0, 0
        all_scores, all_labels = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader):
                int_features, cat_features, labels = batch['dense_features'].to(device), batch['sparse_features'].to(device), batch['label'].to(device)

                outputs = model(int_features, cat_features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_scores.append(outputs.clone().cpu())
                all_labels.append(labels.clone().cpu())
            all_scores = torch.cat(all_scores, dim=-1)
            all_labels = torch.cat(all_labels, dim=-1)


        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100*correct/total:.2f}%, '
              f'Val ROC AUC: {roc_auc_score(all_labels, all_scores)}')

In [24]:
input_size = max(dense_transform.n_bins) * len(CriteoDatasetUtils.INT_COLS) + num_hashes * embedding_size * len(CriteoDatasetUtils.CAT_COLS)
model = DCNV2(
    embedding_size=embedding_size,
    cross_layers=3,
    deep_units=[1024, 1024, 1024],
    input_size=input_size,
    cardinality=cardinality,
)
print(input_size)

5486


In [25]:
train_model(model, train_loader, val_loader, epochs=1, lr=1e-4)

100%|██████████| 160/160 [01:10<00:00,  2.27it/s]


Epoch 1/1, Train Loss: 0.4658, Val Loss: 0.4616, Accuracy: 77.83%, Val ROC AUC: 0.7907098869750834


## 6. Сравниваем с катбустом на том же наборе признаков

In [8]:
import catboost as cb

X_train = train_df.drop('label').to_pandas()
y_train = train_df['label'].to_pandas()
X_test = test_df.drop('label').to_pandas()
y_test = test_df['label'].to_pandas()

train_pool = cb.Pool(X_train, y_train, cat_features=CriteoDatasetUtils.CAT_COLS)
test_pool = cb.Pool(X_test, y_test, cat_features=CriteoDatasetUtils.CAT_COLS)

model = cb.CatBoostClassifier(
    iterations=1000,
    loss_function="Logloss",
    eval_metric="AUC",
    early_stopping_rounds=50,
    task_type="GPU",
    devices='1'   
)
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

Learning rate set to 0.035238


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7171938	best: 0.7171938 (0)	total: 537ms	remaining: 8m 56s
1:	total: 844ms	remaining: 7m 1s
2:	total: 1.17s	remaining: 6m 30s
3:	total: 1.4s	remaining: 5m 49s
4:	total: 1.63s	remaining: 5m 25s
5:	test: 0.7310488	best: 0.7310488 (5)	total: 2.13s	remaining: 5m 52s
6:	total: 2.29s	remaining: 5m 24s
7:	total: 2.53s	remaining: 5m 13s
8:	total: 2.78s	remaining: 5m 6s
9:	total: 3.02s	remaining: 4m 58s
10:	test: 0.7424860	best: 0.7424860 (10)	total: 3.33s	remaining: 4m 59s
11:	total: 3.57s	remaining: 4m 54s
12:	total: 4.09s	remaining: 5m 10s
13:	total: 4.33s	remaining: 5m 4s
14:	total: 4.56s	remaining: 4m 59s
15:	test: 0.7461342	best: 0.7461342 (15)	total: 4.74s	remaining: 4m 51s
16:	total: 4.94s	remaining: 4m 45s
17:	total: 5.39s	remaining: 4m 53s
18:	total: 5.74s	remaining: 4m 56s
19:	total: 5.95s	remaining: 4m 51s
20:	test: 0.7499118	best: 0.7499118 (20)	total: 6.21s	remaining: 4m 49s
21:	total: 6.42s	remaining: 4m 45s
22:	total: 6.63s	remaining: 4m 41s
23:	total: 6.95s	remaining

Из [DCN V2: Improved Deep & Cross Network and Practical Lessons for Web-scale Learning to Rank Systems](https://arxiv.org/abs/2008.13535):
<div style="width:50%; margin: auto;">

![](https://i.ibb.co/HDHJ8Nzq/level-improvement.png)
![](https://i.ibb.co/fYpyrKBs/table.png)

</div>

## 7. Применяем полученные знания в YSDA-RecSys-2025 Lavka

- Заменяем FFN на ResNet или DenseNet с лекции.
- Перебираем гиперпараметры (размер модели, batch_size, lr, ...).
- Ускоряем обучения за счет DCN-Mix (см. [DCN V2: Improved Deep & Cross Network and Practical Lessons for Web-scale Learning to Rank Systems](https://arxiv.org/abs/2008.13535)).